In [20]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
print('Pytorch version :', torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
print('device :', device)

Pytorch version : 1.7.1
device : cuda:0


In [6]:
train_data = datasets.MNIST('./data',train=True,transform=transforms.ToTensor(),download=False)
test_data = datasets.MNIST('./data',train=False,transform=transforms.ToTensor(),download=False)

In [8]:
BATCH_SIZE = 256
train_iter = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)
test_iter = DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)

In [21]:
class MLP(nn.Module):
    def __init__(self, x_dim, h_dim, y_dim):
        super(MLP, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.y_dim = y_dim
        self.lin_1 = nn.Linear(x_dim, h_dim)
        self.lin_2 = nn.Linear(h_dim, y_dim)
        self.init_param()
        
    def init_param(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.kaiming_normal_(param)
            else:
                nn.init.zeros_(param)
    
    def forward(self, x):
        x = self.lin_2(F.relu(self.lin_1(x)))
        return nn.Softmax(x)

In [26]:
model = MLP(x_dim=784, h_dim=256, y_dim=10).to(device)

In [27]:
for name, param in model.named_parameters():
    print(name, param)

lin_1.weight Parameter containing:
tensor([[ 0.0550,  0.0130, -0.0717,  ..., -0.0430,  0.0790, -0.0781],
        [-0.0472, -0.0671, -0.1243,  ..., -0.0626,  0.0265,  0.0170],
        [ 0.0197,  0.0420, -0.0035,  ..., -0.0070,  0.0761, -0.0688],
        ...,
        [ 0.0074,  0.0235,  0.0645,  ...,  0.0329,  0.1075, -0.0202],
        [ 0.0350, -0.0392, -0.0007,  ..., -0.0509,  0.0385, -0.0209],
        [ 0.0231, -0.0434,  0.0342,  ...,  0.0130, -0.0198,  0.0524]],
       device='cuda:0', requires_grad=True)
lin_1.bias Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.

In [28]:
def evaluation(data_iter,model,device):
    for batch_in, batch_label in data_iter:
        x = batch_in.view(-1,28*28).to(device)
        y = batch_label.to(device)
        y_predict = model.forward(x)
        _, indices = torch.max(y_predict, dim=-1)
        

In [29]:
evaluation(train_iter,model,device)

Softmax(
  dim=tensor([[-0.6838,  0.3310,  0.5917,  ..., -0.0870,  0.6486, -0.0334],
          [-0.3283,  0.6309,  0.3565,  ...,  0.0298,  0.6258, -0.2862],
          [-0.7445,  0.8789,  0.7630,  ...,  0.2059,  0.8048,  0.3830],
          ...,
          [-1.0277,  0.9990,  0.3657,  ...,  0.1947,  0.2932, -0.0283],
          [-0.3907,  0.2720,  0.7960,  ...,  0.0204,  0.0867,  0.2598],
          [-1.0766,  0.1647,  0.5466,  ..., -0.4222,  0.7864, -0.1261]],
         device='cuda:0', grad_fn=<AddmmBackward>)
) tensor([9, 6, 6, 4, 8, 8, 9, 1, 6, 4, 9, 9, 1, 9, 5, 5, 9, 2, 1, 5, 1, 9, 9, 3,
        6, 8, 5, 2, 4, 3, 6, 2, 3, 5, 7, 1, 9, 2, 3, 9, 8, 9, 2, 8, 3, 5, 8, 1,
        4, 3, 5, 3, 7, 3, 0, 4, 7, 1, 9, 1, 9, 8, 1, 2, 6, 2, 7, 0, 7, 5, 8, 3,
        1, 7, 3, 3, 6, 0, 2, 6, 2, 6, 9, 0, 9, 8, 3, 8, 0, 0, 0, 6, 7, 7, 0, 8,
        4, 4, 4, 6, 6, 4, 9, 5, 0, 4, 5, 3, 4, 6, 1, 8, 1, 1, 8, 0, 2, 8, 3, 6,
        4, 3, 0, 3, 0, 4, 6, 2, 6, 7, 0, 5, 1, 0, 9, 7, 9, 9, 6, 6, 0, 8, 1, 4,
      

) tensor([0, 4, 6, 2, 7, 1, 5, 7, 3, 5, 3, 0, 8, 0, 4, 9, 4, 3, 1, 0, 2, 6, 2, 7,
        0, 9, 1, 7, 8, 5, 6, 9, 4, 3, 8, 8, 5, 6, 8, 9, 3, 6, 3, 4, 9, 6, 1, 9,
        5, 2, 2, 0, 1, 4, 0, 1, 4, 4, 9, 4, 6, 8, 7, 2, 4, 2, 2, 3, 1, 1, 0, 2,
        3, 2, 0, 5, 9, 4, 7, 5, 7, 4, 9, 8, 3, 5, 7, 9, 3, 1, 4, 6, 6, 3, 2, 6,
        8, 6, 9, 0, 1, 7, 5, 4, 8, 0, 9, 9, 0, 5, 2, 6, 9, 0, 8, 5, 8, 0, 9, 8,
        4, 5, 4, 8, 8, 0, 3, 9, 8, 9, 2, 4, 3, 6, 5, 5, 1, 4, 9, 6, 0, 5, 3, 1,
        8, 6, 7, 8, 4, 1, 0, 9, 3, 8, 2, 1, 7, 3, 2, 9, 8, 6, 6, 4, 0, 8, 6, 8,
        6, 8, 3, 0, 5, 3, 0, 5, 8, 2, 8, 3, 6, 1, 7, 0, 8, 0, 1, 9, 7, 6, 9, 3,
        3, 5, 7, 1, 1, 2, 7, 8, 9, 6, 2, 2, 5, 1, 5, 6, 1, 8, 9, 6, 5, 5, 6, 8,
        5, 6, 6, 4, 5, 8, 9, 0, 3, 0, 6, 9, 1, 9, 8, 5, 8, 7, 5, 2, 2, 0, 1, 4,
        5, 0, 4, 3, 7, 7, 2, 7, 9, 6, 5, 2, 7, 8, 6, 9], device='cuda:0')
Softmax(
  dim=tensor([[-0.7500,  0.8598,  0.4259,  ...,  0.5049,  0.3378, -0.1787],
          [-0.4837,  0.8943,  0.1937,  

) tensor([5, 5, 7, 0, 2, 0, 5, 3, 7, 0, 3, 3, 2, 3, 7, 4, 2, 4, 8, 7, 7, 0, 3, 0,
        5, 8, 6, 1, 5, 2, 2, 9, 5, 9, 1, 9, 2, 4, 4, 6, 9, 1, 2, 2, 6, 5, 4, 3,
        4, 3, 9, 4, 6, 2, 1, 2, 3, 9, 8, 9, 6, 3, 4, 7, 8, 0, 1, 1, 3, 4, 6, 8,
        1, 9, 7, 8, 4, 7, 7, 0, 4, 9, 1, 5, 5, 5, 4, 2, 9, 8, 1, 7, 4, 6, 4, 9,
        2, 1, 7, 9, 4, 8, 5, 5, 2, 2, 3, 2, 0, 9, 3, 5, 6, 8, 5, 8, 0, 5, 8, 2,
        2, 1, 1, 6, 3, 9, 7, 9, 9, 7, 7, 3, 2, 9, 6, 5, 4, 5, 8, 4, 5, 3, 8, 2,
        8, 2, 5, 8, 8, 1, 4, 8, 6, 4, 0, 6, 4, 9, 3, 4, 9, 7, 1, 7, 9, 0, 4, 0,
        6, 4, 1, 2, 2, 8, 3, 3, 7, 4, 9, 6, 2, 9, 0, 2, 7, 2, 3, 2, 9, 1, 9, 3,
        5, 2, 2, 4, 7, 3, 6, 9, 9, 3, 3, 4, 2, 4, 6, 6, 7, 1, 3, 1, 1, 2, 3, 1,
        7, 0, 8, 1, 7, 0, 5, 1, 9, 0, 0, 2, 2, 8, 4, 8, 7, 2, 4, 1, 7, 3, 3, 6,
        1, 8, 6, 3, 9, 0, 3, 5, 2, 8, 0, 6, 3, 1, 5, 5], device='cuda:0')
Softmax(
  dim=tensor([[-0.5765,  0.6879,  0.5045,  ...,  0.3641,  0.9477, -0.1055],
          [-0.2793,  0.3467,  0.4879,  

) tensor([2, 5, 6, 3, 9, 7, 4, 2, 3, 1, 5, 6, 1, 5, 9, 9, 4, 9, 4, 6, 9, 4, 2, 4,
        3, 3, 7, 1, 3, 1, 4, 1, 3, 1, 2, 0, 3, 6, 8, 0, 8, 6, 4, 4, 0, 2, 0, 0,
        9, 6, 4, 2, 0, 2, 9, 2, 1, 0, 6, 1, 2, 1, 4, 3, 5, 2, 8, 3, 9, 1, 3, 7,
        4, 1, 7, 6, 7, 3, 7, 5, 7, 6, 2, 2, 6, 8, 9, 1, 1, 9, 3, 3, 5, 3, 8, 2,
        2, 0, 9, 3, 6, 0, 9, 1, 0, 0, 8, 5, 1, 1, 3, 5, 0, 7, 9, 9, 5, 7, 9, 7,
        6, 3, 9, 8, 1, 1, 0, 8, 0, 0, 2, 0, 0, 6, 4, 4, 9, 0, 8, 4, 3, 6, 8, 0,
        1, 7, 5, 1, 3, 7, 1, 2, 9, 3, 9, 1, 2, 0, 3, 2, 6, 3, 6, 1, 2, 1, 9, 2,
        3, 1, 7, 2, 6, 4, 5, 5, 1, 3, 8, 3, 0, 6, 2, 7, 6, 6, 9, 8, 3, 8, 5, 5,
        6, 9, 0, 5, 3, 2, 1, 6, 3, 6, 0, 6, 3, 6, 1, 2, 3, 5, 2, 9, 1, 3, 2, 8,
        0, 2, 4, 4, 0, 1, 8, 5, 4, 4, 2, 6, 2, 9, 8, 9, 5, 6, 1, 3, 4, 0, 3, 3,
        0, 2, 0, 3, 0, 8, 6, 4, 7, 9, 1, 1, 7, 8, 6, 5], device='cuda:0')
Softmax(
  dim=tensor([[-1.0027,  0.4796,  0.2547,  ...,  0.4936,  0.4757,  0.3081],
          [-1.2534,  0.7038,  0.9116,  

Softmax(
  dim=tensor([[-0.2717,  0.6778,  1.0017,  ...,  0.0075,  0.7912,  0.4029],
          [-1.2056,  0.4231,  0.6860,  ...,  0.4002,  1.2093,  0.3916],
          [-0.9191,  0.6569, -0.2015,  ..., -0.1717,  0.4217, -0.3322],
          ...,
          [-0.6666,  0.2774,  0.6470,  ..., -0.0352,  0.4634,  0.4499],
          [-0.6529,  0.5717,  0.2815,  ..., -0.1952,  0.7153,  0.0616],
          [-1.0896,  0.5206,  0.6041,  ...,  0.1558,  1.1752,  0.7149]],
         device='cuda:0', grad_fn=<AddmmBackward>)
) tensor([1, 3, 3, 8, 7, 6, 1, 7, 8, 1, 1, 1, 2, 6, 6, 8, 3, 6, 1, 1, 6, 6, 7, 3,
        9, 8, 2, 9, 7, 7, 2, 5, 1, 3, 5, 4, 6, 9, 9, 2, 5, 8, 0, 0, 0, 8, 7, 1,
        8, 9, 6, 4, 0, 9, 9, 6, 1, 4, 1, 0, 3, 3, 1, 8, 4, 4, 5, 6, 2, 3, 4, 7,
        5, 3, 5, 0, 1, 9, 0, 2, 8, 7, 3, 0, 2, 4, 0, 6, 8, 9, 9, 8, 5, 5, 2, 5,
        5, 4, 9, 5, 1, 7, 8, 3, 1, 2, 3, 2, 5, 6, 7, 7, 4, 5, 8, 0, 1, 6, 3, 9,
        2, 2, 7, 3, 2, 8, 1, 9, 6, 6, 9, 0, 5, 7, 3, 0, 1, 4, 0, 7, 6, 1, 5, 4,
      

...

) tensor([1, 2, 4, 8, 4, 9, 0, 7, 1, 2, 0, 9, 6, 3, 8, 6, 0, 6, 9, 3, 5, 5, 8, 8,
        2, 2, 3, 5, 9, 6, 5, 9, 1, 6, 2, 2, 1, 3, 4, 7, 4, 9, 4, 3, 4, 4, 4, 3,
        3, 3, 3, 4, 7, 6, 4, 4, 0, 1, 1, 7, 1, 3, 9, 6, 8, 5, 7, 4, 2, 2, 3, 5,
        0, 9, 3, 0, 1, 8, 2, 0, 2, 6, 6, 4, 0, 8, 0, 9, 3, 6, 1, 7, 8, 9, 2, 3,
        4, 0, 0, 9, 3, 7, 4, 0, 7, 7, 2, 0, 6, 4, 6, 6, 0, 2, 8, 6, 3, 9, 0, 9,
        1, 0, 8, 1, 5, 2, 5, 1, 9, 7, 3, 8, 5, 4, 8, 8, 0, 9, 1, 5, 7, 0, 7, 1,
        1, 8, 0, 4, 1, 5, 2, 5, 8, 5, 7, 4, 4, 3, 0, 0, 3, 0, 8, 8, 4, 5, 5, 6,
        0, 1, 1, 2, 2, 5, 0, 1, 6, 9, 5, 7, 4, 1, 4, 0, 3, 2, 4, 6, 3, 8, 4, 1,
        7, 2, 6, 3, 5, 8, 0, 8, 7, 5, 3, 9, 9, 0, 7, 9, 8, 5, 2, 4, 7, 6, 8, 8,
        6, 4, 4, 7, 6, 5, 5, 3, 6, 2, 0, 1, 7, 5, 0, 2, 8, 3, 3, 4, 1, 0, 4, 5,
        4, 1, 0, 0, 1, 0, 6, 1, 4, 2, 8, 0, 4, 2, 8, 3], device='cuda:0')
Softmax(
  dim=tensor([[-0.4947,  0.5576,  0.7582,  ...,  0.4449,  0.7066,  0.1868],
          [-0.4921,  0.1166,  0.2605,  

) tensor([1, 0, 7, 2, 1, 2, 0, 7, 2, 1, 3, 9, 7, 5, 5, 6, 3, 4, 3, 7, 1, 6, 4, 5,
        0, 5, 8, 9, 4, 8, 2, 2, 6, 4, 7, 9, 9, 1, 7, 5, 4, 7, 3, 2, 5, 1, 3, 5,
        8, 6, 1, 9, 1, 9, 1, 4, 3, 7, 2, 9, 6, 3, 2, 8, 2, 8, 8, 3, 4, 1, 0, 2,
        1, 2, 4, 7, 3, 5, 7, 8, 9, 6, 3, 7, 9, 8, 7, 7, 8, 7, 3, 2, 3, 7, 6, 6,
        4, 9, 4, 3, 6, 4, 2, 7, 8, 3, 2, 1, 1, 3, 3, 8, 5, 8, 1, 4, 2, 0, 0, 9,
        6, 0, 3, 9, 8, 0, 7, 4, 9, 1, 9, 8, 6, 0, 5, 9, 6, 4, 6, 8, 4, 0, 7, 5,
        4, 5, 4, 5, 8, 4, 8, 9, 1, 4, 0, 5, 1, 8, 0, 0, 4, 1, 7, 8, 7, 5, 2, 6,
        0, 3, 9, 7, 4, 9, 3, 3, 6, 9, 8, 7, 2, 5, 8, 7, 0, 0, 7, 9, 7, 1, 1, 2,
        0, 7, 0, 2, 0, 1, 8, 2, 7, 3, 3, 5, 7, 8, 1, 1, 9, 4, 3, 2, 6, 2, 7, 1,
        0, 1, 6, 7, 1, 5, 4, 8, 3, 1, 3, 1, 6, 6, 5, 8, 1, 9, 1, 4, 4, 3, 7, 0,
        4, 5, 6, 1, 1, 3, 9, 7, 1, 0, 0, 1, 4, 9, 1, 4], device='cuda:0')
Softmax(
  dim=tensor([[-0.6284,  0.6794,  0.5779,  ..., -0.0710,  1.2402, -0.0895],
          [-0.8192,  0.9425,  0.9017,  

) tensor([0, 1, 7, 1, 2, 8, 8, 0, 9, 9, 1, 7, 0, 4, 4, 4, 6, 9, 5, 3, 7, 5, 7, 8,
        8, 0, 9, 8, 0, 7, 3, 2, 1, 4, 9, 4, 5, 2, 2, 6, 8, 4, 4, 6, 4, 7, 4, 5,
        2, 6, 9, 5, 8, 5, 7, 2, 6, 0, 8, 8, 4, 2, 1, 7, 7, 5, 4, 6, 4, 0, 1, 3,
        6, 9, 5, 1, 8, 6, 4, 1, 7, 4, 5, 2, 7, 0, 5, 8, 1, 0, 5, 3, 1, 6, 6, 0,
        6, 8, 5, 1, 9, 8, 1, 6, 3, 9, 1, 1, 7, 1, 1, 4, 3, 8, 6, 5, 6, 1, 3, 5,
        7, 5, 5, 2, 4, 5, 0, 5, 7, 8, 7, 9, 7, 2, 3, 0, 4, 6, 3, 7, 5, 8, 7, 6,
        9, 6, 9, 6, 3, 6, 3, 9, 5, 8, 0, 2, 2, 9, 3, 5, 2, 0, 9, 6, 6, 1, 2, 2,
        1, 7, 3, 3, 4, 0, 2, 9, 7, 5, 7, 6, 6, 7, 6, 7, 5, 4, 9, 7, 9, 2, 5, 0,
        8, 2, 2, 1, 9, 3, 4, 9, 1, 6, 8, 7, 8, 4, 8, 5, 7, 6, 5, 5, 1, 7, 1, 1,
        0, 3, 9, 1, 2, 5, 3, 6, 4, 0, 7, 6, 6, 9, 8, 8, 5, 0, 0, 8, 9, 1, 0, 1,
        0, 6, 3, 8, 6, 9, 8, 9, 4, 1, 2, 1, 8, 2, 4, 7], device='cuda:0')
Softmax(
  dim=tensor([[-0.7625,  0.3327,  0.1062,  ...,  0.2148,  0.4599, -0.1259],
          [-0.5141,  0.5786, -0.0123,  

) tensor([8, 4, 1, 3, 3, 5, 9, 0, 9, 8, 4, 3, 3, 6, 4, 6, 3, 7, 0, 7, 1, 7, 1, 5,
        6, 8, 4, 6, 4, 9, 3, 6, 5, 7, 1, 4, 1, 1, 9, 4, 4, 7, 4, 5, 7, 9, 5, 7,
        4, 4, 6, 7, 6, 8, 3, 0, 6, 8, 7, 5, 8, 6, 3, 5, 4, 2, 9, 6, 6, 6, 8, 5,
        6, 8, 7, 2, 2, 2, 9, 4, 1, 4, 9, 6, 7, 4, 4, 7, 1, 5, 4, 7, 4, 2, 1, 0,
        7, 4, 1, 7, 5, 2, 7, 5, 2, 4, 8, 7, 8, 5, 0, 8, 6, 7, 8, 9, 8, 1, 8, 1,
        4, 2, 2, 2, 7, 2, 7, 9, 3, 3, 1, 2, 6, 4, 9, 6, 8, 8, 8, 1, 9, 7, 5, 0,
        7, 3, 3, 5, 7, 2, 8, 8, 1, 1, 7, 2, 4, 0, 7, 0, 4, 7, 5, 1, 5, 5, 7, 8,
        4, 7, 9, 9, 5, 9, 6, 2, 3, 6, 8, 9, 7, 4, 2, 4, 2, 3, 2, 2, 5, 1, 2, 0,
        2, 5, 0, 1, 6, 9, 9, 6, 4, 3, 8, 8, 2, 9, 1, 7, 1, 6, 9, 1, 0, 2, 2, 1,
        3, 6, 5, 2, 5, 4, 2, 2, 4, 5, 9, 1, 7, 7, 3, 1, 7, 2, 7, 9, 8, 8, 0, 9,
        4, 5, 3, 0, 1, 2, 0, 8, 4, 6, 2, 3, 3, 7, 7, 4], device='cuda:0')
Softmax(
  dim=tensor([[-0.9887,  1.0526,  0.6536,  ...,  0.4098,  0.6424,  0.1296],
          [-0.9050,  0.6413,  0.7529,  

Softmax(
  dim=tensor([[-7.1088e-01,  9.5920e-01,  1.0567e+00,  4.2680e-01,  4.4589e-01,
            2.2144e-01,  1.2056e-02,  1.8143e-01,  9.0958e-01,  6.0839e-01],
          [-5.3516e-01,  7.6191e-01,  4.9337e-01, -3.9698e-01,  5.7932e-01,
           -1.3407e-02,  6.5970e-02,  2.6507e-01,  9.2952e-01,  5.8507e-02],
          [-8.3201e-01,  9.5836e-01,  1.4295e-01,  7.2273e-01, -5.4530e-01,
            8.8747e-02, -3.8158e-02,  3.2404e-01,  6.5928e-01, -4.1097e-01],
          [-2.8033e-01,  6.2647e-01,  6.0917e-01,  2.3754e-01, -5.3750e-01,
            7.4221e-02, -9.1947e-02,  2.4070e-01,  8.2928e-01,  5.1299e-01],
          [-5.3892e-01,  4.5337e-01,  4.5746e-01,  8.8502e-02,  2.5214e-01,
            4.2607e-02, -4.0129e-01,  5.0772e-02,  6.1074e-01,  1.2619e-01],
          [-3.7830e-01,  6.4723e-01,  4.9888e-01,  4.3075e-01,  1.6821e-01,
            6.1384e-01, -1.9434e-01,  7.7192e-02,  3.0234e-01, -2.7993e-01],
          [-1.4802e+00,  5.3370e-01,  6.0265e-01,  4.9472e-01,  5.199

In [47]:
a = 0x0011

In [45]:
b = 0b0101

In [41]:
print(a)

b'\x00x0011'


In [39]:
a

b'\x00b0011'

In [42]:
c = 'abcd dbad asbdfds dasfb'

In [44]:
c.replace('d','D',100)

'abcD DbaD asbDfDs Dasfb'

In [50]:
d = bin(a+b)

In [53]:
a+eval(d)

39